In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [ ]:
import os
import tensorflow as tf
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
#from jupyterthemes import jtplot

import nmca_model
from TwoChannelModel import TwoChannelModel
from correlation_analysis import CCA, PCC_Matrix
from plot import plot_eval
from tf_summary import write_scalar_summary, write_image_summary, write_PCC_summary

LOGPATH = '/var/tmp/mkuschel/tf_logs/'

# Generate Data

In [ ]:
data_model = TwoChannelModel(num_samples=1000)
y_1, y_2, Az_1, Az_2, z_1, z_2 = data_model('Parabola')

In [ ]:
TwoChannelModel.plot_shared_components(z_1, z_2)

In [ ]:
TwoChannelModel.plot_non_linearities(y_1, y_2, Az_1, Az_2)

# Build model

In [ ]:
model = nmca_model.build_nmca_model(hdim=256)

In [ ]:
model.summary()

# Train one network

In [ ]:
#writer, folder = nmca_model.create_writer(LOGPATH)
optimizer = tf.keras.optimizers.Adam()

lambda_reg = 1e-10
shared_dim = 0
sum_of_grads = list()
t_ccor = [1,1,0,0,0]

for epoch in tqdm(range(100000), desc='Epochs'):
    if epoch%20000 == 0:
            if shared_dim < 4:
                shared_dim += 1
                print(str(shared_dim)+" shared dimensions")

    with tf.GradientTape() as tape:
        # Watch the input to be able to compute the gradient later
        tape.watch([y_1, y_2])
        # Forward path
        [fy_1, fy_2], [yhat_1, yhat_2] = model([tf.transpose(y_1), tf.transpose(y_2)])
        # Loss computation
        loss, cca_loss, rec_loss, ccor = nmca_model.compute_loss(y_1, y_2, fy_1, fy_2, yhat_1, yhat_2, shared_dim, lambda_reg=lambda_reg)
        
        # Compute gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        sum_of_grads.append(np.sum([np.linalg.norm(grad) for grad in gradients]))

        if epoch % 10 == 0:
            B1, B2, epsilon, omega, ccor = CCA(fy_1, fy_2, 5)
            if t_ccor[0] == 1:
                sim_v1 = nmca_model.compute_similarity_metric_v1(S=z_1[:2], U=0.5*(omega+epsilon))
            else:
                sim_v1 = 0
            sim_v2 = nmca_model.compute_similarity_metric_v1(S=z_1[:2], U=epsilon)
            sim_v3 = nmca_model.compute_similarity_metric_v1(S=z_2[:2], U=omega)
            dist = nmca_model.compute_distance_metric(S=z_1[:2], U=0.5 * (omega + epsilon)[:2])

            write_scalar_summary(
                writer=writer, 
                epoch=epoch, 
                list_of_tuples=[
                    (np.sum(sum_of_grads), 'Gradients/Sum'),
                    (loss, 'Loss/Total'),
                    (cca_loss, 'Loss/CCA'),
                    (rec_loss, 'Loss/reconstruction'),
                    (ccor[0], 'Canonical correlation/0'),
                    (ccor[1], 'Canonical correlation/1'),
                    (ccor[2], 'Canonical correlation/2'),
                    (ccor[3], 'Canonical correlation/3'),
                    (ccor[4], 'Canonical correlation/4'),
                    (dist, 'Performance Measures/Distance measure'),
                    (sim_v1, 'Performance Measures/Similarity measure'),
                    (sim_v2, 'Performance Measures/Similarity measure 1st view'),
                    (sim_v3, 'Performance Measures/Similarity measure 2nd view'),
                ]
            )
            sum_of_grads = list()

        if epoch % 5000 == 0:
            write_image_summary(writer, epoch, Az_1, Az_2, y_1, y_2, fy_1, fy_2, yhat_1, yhat_2)
            write_PCC_summary(writer, epoch, z_1, z_2, epsilon, omega, 1000)
            
    
    # Backpropagate through network
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
model.save(log_folder)

# Eval network

In [ ]:
# Forward path
[fy_1, fy_2], [yhat_1, yhat_2] = model([tf.transpose(y_1), tf.transpose(y_2)])

# Compute CCA
B1, B2, epsilon, omega, ccor = CCA(fy_1, fy_2, 2)

fy_1, fy_2 = tf.transpose(fy_1), tf.transpose(fy_2)
yhat_1, yhat_2 = tf.transpose(yhat_1), tf.transpose(yhat_2)

In [ ]:
S = z_1[:2]
Ps = np.eye(1000) - tf.transpose(S)@np.linalg.inv(S@tf.transpose(S))@S
U = 0.5*(omega+epsilon)
Q = scipy.linalg.orth(tf.transpose(U))
dist = np.linalg.norm(Ps@Q, ord=2)
print("Dist: "+str(dist))

In [ ]:
plot_eval(z_1, z_2, Az_1, Az_2, y_1, y_2, fy_1, fy_2, yhat_1, yhat_2, epsilon, omega)